In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from OptWind import windOpt_linear
from OptJoint import jointOpt_linear
from OptStorage import storageOpt_linear
from OptJoint import jointOpt_linear_noArb
from OptDisjoint import disjointOpt_linear
import matplotlib.pyplot as plt

In [2]:
res_profile = np.load('Interresult\solar_winter.npz')['data']
print(res_profile)

[1.30000000e-02 1.30000000e-02 1.31111111e-02 1.30000000e-02
 9.00000000e-03 6.88888889e-03 5.77777778e-03 1.08033333e+00
 1.93971111e+01 4.28098889e+01 4.83293333e+01 4.97325556e+01
 4.96512222e+01 4.98350000e+01 4.99392222e+01 4.80010000e+01
 3.83207778e+01 1.39814444e+01 6.94222222e-01 2.18888889e-02
 1.32222222e-02 1.33333333e-02 1.32222222e-02 1.31111111e-02]


In [3]:
netload_profile = np.load('Interresult\loadnet_winter.npz')['data']
print(netload_profile)

[29.28650562 28.75366933 28.68740456 29.00351878 29.91310533 31.86065556
 34.71766644 36.32446511 35.18510756 33.60811267 32.52607689 31.38863311
 30.46216211 29.77439911 29.25993722 29.19261544 30.60712933 34.46158233
 36.82310422 36.09541933 35.08137889 33.821181   31.97127933 30.23990722]


#### \alpha and \beta

In [4]:
T = len(netload_profile)
alpha = np.zeros(T)
beta_base = np.zeros(T)
for i, d in enumerate(netload_profile):
    if d>=10.9 and d<=51.0:
        alpha[i] = 25.94 + 0.12 *d
        beta_base[i] = 0.12 
    elif d> 51.0 and d<= 58.0:
        alpha[i] = -82.25 + 2.28 *d
        beta_base[i] = 2.28
    elif d>=58.0 and d<= 67.8:
        alpha[i] =  -0.47 + 0.8 *d
        beta_base[i] = 0.8
    else:
        print('ERROR!')
print(alpha)
print(beta_base)

[29.45438067 29.39044032 29.38248855 29.42042225 29.52957264 29.76327867
 30.10611997 30.29893581 30.16221291 29.97297352 29.84312923 29.70663597
 29.59545945 29.51292789 29.45119247 29.44311385 29.61285552 30.07538988
 30.35877251 30.27145032 30.14976547 29.99854172 29.77655352 29.56878887]
[0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12
 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12]


In [5]:
beta= np.diag(beta_base)

#### Exploring possibility

In [6]:
storage_choice = np.arange(0.2, 2, 0.2)
RES_choice = np.arange(0.2, 2, 0.2)
n_storage = len(storage_choice)
n_RES = len(RES_choice)

In [7]:
storage_choice = np.arange(0.2, 2, 0.2)
RES_choice = np.arange(0.2, 2, 0.2)
n_storage = len(storage_choice)
n_RES = len(RES_choice)

In [8]:
m_joint_record = np.zeros((n_storage, n_RES))
m_disjoint_record = np.zeros((n_storage, n_RES))
w_single_record = np.zeros((n_storage, n_RES))
m_joint_record_noArb = np.zeros((n_storage, n_RES))
valueWind_record = np.zeros((n_storage, n_RES)) 
valueWind_record_noArb = np.zeros((n_storage, n_RES)) 
value_arbitrager_record = np.zeros((n_storage, n_RES))

In [9]:
#### Example: storage 1GW, solar 1GW
for i, s_cap in enumerate(storage_choice):
        for j, w_cap in enumerate(RES_choice):
                s_ub = s_cap* np.ones(T)
                s_ub[-1] = s_ub[-1] - 0.5*s_cap
                s_lb = 0* np.ones(T)
                s_lb[-1] = s_lb[-1] + 0.5*s_cap
                s_init = np.zeros(T)
                s_init[0] = 0.5 * s_cap
                eta_c = 1
                eta_d = 1

                # w_cap = 1
                w_true = res_profile * w_cap /100

                w, x, s, m = jointOpt_linear(T, alpha, beta, w_true, s_ub, s_lb, s_init, eta_c, eta_d)
                w_n, x_n, s_n, m_n = jointOpt_linear_noArb(T, alpha, beta, w_true, s_ub, s_lb, s_init, eta_c, eta_d)
                w2_linear, m2_linear = windOpt_linear(T, alpha, beta, w_true)
                valueWind_record[i,j] = m-m2_linear
                valueWind_record_noArb[i,j] = m_n-m2_linear
                m_joint_record[i,j] = m
                w_single_record[i,j]= m2_linear
                m_joint_record_noArb[i,j] = m_n
                eps = 1e-3
                kUb = 10
                w_sol_try, x_sol_try, m_disjoint_wind_try, m_disjoint_storage_try=disjointOpt_linear(T, alpha, beta, w_true, s_ub, s_lb, s_init, eta_c, eta_d, eps, kUb)
                m_disjoint_record[i,j] = m_disjoint_wind_try+ m_disjoint_storage_try
                value_arbitrager_record[i,j] = m_disjoint_storage_try

# print("profit", m_joint_record, m_disjoint_record)
# print("Value", valueWind_record, valueWind_record_noArb, value_arbitrager_record)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-27


In [ ]:
np.savez('.\Interresult\heatmap_solar_winter.npz', 
         m_joint = m_joint_record,
         m_disjoint = m_disjoint_record,
         m_single = w_single_record,
         m_joint_noArb = m_joint_record_noArb,
         valueWind = valueWind_record,
         valueWind_noArb =valueWind_record_noArb,
         value_arbitrager = value_arbitrager_record)